In [5]:
import os, sys
import pandas as pd
from datetime import date

todays_date = date.today().strftime("%Y-%m-%d")

In [31]:
import parsl
from parsl.app.app import python_app

In [7]:
import utility_functions

In [39]:
def localParslConfig_threadpool(params):
 
    import parsl
    from parsl.executors import ThreadPoolExecutor
    from parsl.config import Config

    import os
    workingdir = params['working_dir']
    rundir = os.path.join(workingdir, 'runinfo')
    parsl.clear()
    local_tpex = Config(
        executors=[
            ThreadPoolExecutor(
                label="tpex_Local",
                max_threads=8,
                thread_name_prefix='tpex_run',
                working_dir=workingdir,
            )
        ],
        strategy=None,
        run_dir=rundir
    )

    return(local_tpex)

In [9]:
enformer_predictions_path = "/lus/grand/projects/covid-ct/imlab/users/temi/projects/TFXcan/enformer_pipeline/enformer_predictions/kawakami/predictions_2022-12-20/kawakami" #sys.argv[1] 
log_path = "/lus/grand/projects/covid-ct/imlab/users/temi/projects/TFXcan/enformer_pipeline/predictions_log/kawakami/predictions_log_2022-12-20" #sys.argv[2]
each_id = "kawakami" #sys.argv[3] # e.g. "kawakami" or "cistrome"
agg_type = "aggByMean" #sys.argv[4]

print(f'[INFO] Currently on {each_id}')

upstream = list(range(0, 8))
center = [8]
downstream = list(range(9, 17))

base_path = '/grand/projects/covid-ct/imlab/users/temi/projects/TFXcan'
# enformer_predictions_path = f'{base_path}/enformer_pipeline/enformer_predictions/{each_id}_reference/predictions_2022-12-11/{each_id}_FOXA1'
# log_path = f'{base_path}/enformer_pipeline/predictions_log/predictions_log_2022-12-11'

save_dir = f'{base_path}/modeling_pipeline/data/train-test-val/{each_id}/data_{todays_date}'
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)

TF = 'FOXA1'
#data_names = ['aggByCenter']
#data_names = ['aggByMean', 'aggByCenter', 'aggByUpstream', 'aggByDownstream', 'aggByUpstreamDownstream']

logpath = f'{log_path}/{each_id}_{TF}_predictions_log.csv'
log_data = pd.read_csv(logpath)
log_data = log_data.drop_duplicates(subset=['motif']).iloc[1:20, ]

[INFO] Currently on kawakami


In [26]:
def generate_batch(lst, batch_n, len_lst = None):
    """
    Given a list, this function yields batches of an unspecified size but the number of batches is equal to `batch_n`
    E.g. generate_batch([0, 1, 2, 3, 4, 5, 6], batch_n=2) -> (0, 1, 2, 3), (4, 5, 6)
    
    Parameters:
        lst: list
        batch_n: int
            Number of batches to return
        len_lst: None or num (length of the input list)
    Yields
        `batch_n` batches of the list
    """
    import math
    # how many per batch
    if len_lst is not None:
        n_elems = math.ceil(len_lst/batch_n)
    else:
        n_elems = math.ceil(len(lst)/batch_n)
        
    for i in range(0, len(lst), n_elems):
        yield lst[i:(i + n_elems)]

In [40]:
# load parsl
bpath = os.path.join(base_path, 'modeling_pipeline')
localParslConfig_threadpool({'working_dir': bpath})

Config(
    app_cache=True, 
    checkpoint_files=None, 
    checkpoint_mode=None, 
    checkpoint_period=None, 
    executors=[ThreadPoolExecutor(
        label='tpex_Local', 
        managed=True, 
        max_threads=8, 
        storage_access=None, 
        thread_name_prefix='tpex_run', 
        working_dir='/grand/projects/covid-ct/imlab/users/temi/projects/TFXcan/modeling_pipeline'
    )], 
    garbage_collect=True, 
    initialize_logging=True, 
    internal_tasks_max_threads=10, 
    max_idletime=120.0, 
    monitoring=None, 
    retries=0, 
    retry_handler=None, 
    run_dir='/grand/projects/covid-ct/imlab/users/temi/projects/TFXcan/modeling_pipeline/runinfo', 
    strategy=None, 
    usage_tracking=False
)

In [33]:
@python_app
def collect_modeling_data_for_kawakami(each_id, log_data, predictions_path, TF, base_path, save_dir, agg_types, batch_num=None):

    import h5py
    import numpy as np
    import os
    import pandas as pd
    #import tqdm
    # read in one of the files

    try:
        import utility_functions
    except ModuleNotFoundError:
        print(f'[ERROR] Utility_functions module not found.')

    #exec(open(f'{base_path}/modeling_pipeline/scripts/collect_model_data/utility-functions.py').read(), globals(), globals())
    # bpath = os.path.join(base_path, 'modeling_pipeline')
    # localParslConfig_threadpool({'working_dir': bpath})

    kawakami_predictions = {}

    for dt in log_data.loc[log_data['sequence_type'] == 'ref', ].motif.values.tolist():
        fle = f'{predictions_path}/{dt}_predictions.h5'
        if os.path.isfile(fle):
            with h5py.File(fle, 'r') as f:
                filekey = list(f.keys())[0]
                kawakami_predictions[dt] = np.vstack(list(f[filekey]))
        else:
            print(f'[ERROR] {dt} predictions file does not exist.')

    print(f'[INFO] Finished collecting {len(kawakami_predictions)} predictions for {each_id}')

    #dt_aggbycenter = agg_by_center(kawakami_predictions, center=8)
    #data_list = [dt_aggbycenter]

    data_dict = {}
    for agg_type in agg_types:
        if agg_type == 'aggByMean': data_dict[agg_type] = utility_functions.agg_by_mean(kawakami_predictions)
        if agg_type == 'aggByCenter': data_dict[agg_type] = utility_functions.agg_by_center(kawakami_predictions)
        if agg_type == 'aggByUpstream': data_dict[agg_type] = utility_functions.agg_by_mean(kawakami_predictions, use_bins=upstream)
        if agg_type == 'aggByDownstream': data_dict[agg_type] = utility_functions.agg_by_mean(kawakami_predictions, use_bins=downstream)
        if agg_type == 'aggByUpstreamDownstream': data_dict[agg_type] = utility_functions.agg_by_mean(kawakami_predictions, use_bins=upstream + downstream)

    #test_aggbymean, test_aggbycenter, test_aggbymean_upstream, test_aggbymean_downstream, test_aggbymean_upstream_downstream = agg_byall(kawakami_predictions)
    #data_list = [test_aggbymean, test_aggbycenter, test_aggbymean_upstream, test_aggbymean_downstream, test_aggbymean_upstream_downstream]

    for i, agg_type in enumerate(data_dict):

        #ty = pd.concat([pd.Series(list(kawakami_predictions.keys())), pd.DataFrame(dt)], axis=1)
        ty = pd.DataFrame(data_dict[agg_type])
        print(f'[INFO] Dimension of collected data is {ty.shape[0]} by {ty.shape[1]}')

        column_names = ['id']
        column_names.extend([f'f_{i}' for i in range(1, ty.shape[1])])

        #print(len(column_names))
        ty = ty.set_axis(column_names, axis=1, inplace=False)
        print(ty.iloc[0:5, 0:5])

        if batch_num is None:
            ty.to_csv(path_or_buf=f'{save_dir}/{each_id}_{agg_type}_{TF}.csv.gz', index=False, compression='gzip')
        else:
            ty.to_csv(path_or_buf=f'{save_dir}/{each_id}_{agg_type}_{TF}_batch_{batch_num}.csv.gz', index=False, compression='gzip')
            
    print(f'[INFO] Finished saving data for {each_id}')

    return(0)

In [38]:
count = 0
app_futures = []
range_batches = generate_batch(range(0, log_data.shape[0] + 1), batch_n=3)
for range_batch in range_batches:

    collected = collect_modeling_data_for_kawakami(each_id=each_id, log_data=log_data.iloc[list(range_batch), ], predictions_path=enformer_predictions_path, TF=TF, agg_types=[agg_type], base_path=base_path, save_dir=save_dir, batch_num=count)
    app_futures.append(collected)
    count = count + 1

app_execs = [r.result() for r in app_futures]

RuntimeError: Must first load config

In [25]:
collected = collect_modeling_data_for_kawakami(each_id=each_id, log_data=log_data, predictions_path=enformer_predictions_path, TF=TF, agg_types=[agg_type], base_path=base_path, save_dir=save_dir)

[INFO] Finished collecting 19 predictions for kawakami
[INFO] Dimension of collected data is 19 by 5314
                         id          f_1           f_2          f_3  \
0  chr3_192940730_192940738  0.052521303   0.059343856   0.08436744   
1   chr12_38940385_38940393   0.05062969   0.042285476  0.031967606   
2   chr10_31409432_31409440  0.004776896  0.0055661146  0.004850283   
3  chr2_135230278_135230286  0.119184524    0.13753144  0.091157116   
4   chr11_33763385_33763393   0.06338547    0.05521163  0.030234452   

           f_4  
0  0.077099375  
1   0.08609707  
2  0.008029616  
3  0.104358084  
4   0.06341503  


/tmp/ipykernel_117367/2756105060.py:56: FutureWarning: DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `obj = obj.set_axis(..., copy=False)` instead
  ty = ty.set_axis(column_names, axis=1, inplace=False)


[INFO] Finished saving data for kawakami


In [9]:
def collect_modeling_data_for_kawakami(each_id, log_data, predictions_path, TF, data_names, base_path, save_dir):

    import h5py
    import numpy as np
    import os
    import pandas as pd
    import tqdm
    # read in one of the files

    exec(open(f'{base_path}/modeling_pipeline/scripts/collect_model_data/utility-functions.py').read(), globals(), globals())

    kawakami_predictions = {}

    for dt in log_data.loc[log_data['sequence_type'] == 'ref', ].motif.values.tolist():
        fle = f'{predictions_path}/{dt}_predictions.h5'
        print(fle)
        if os.path.isfile(fle):
            with h5py.File(fle, 'r') as f:
                filekey = list(f.keys())[0]
                # should I select tracks? ; maybe not yet
                kawakami_predictions[dt] = np.vstack(list(f[filekey]))
        else:
            print('File does not exist')

    print(f'[INFO] Finished collecting {len(kawakami_predictions)} predictions for {each_id}')

    dt_aggbycenter = agg_by_center(kawakami_predictions, center=8)
    data_list = [dt_aggbycenter]

    # test_aggbymean, test_aggbycenter, test_aggbymean_upstream, test_aggbymean_downstream, test_aggbymean_upstream_downstream = agg_byall(freedman_predictions)
    # data_list = [test_aggbymean, test_aggbycenter, test_aggbymean_upstream, test_aggbymean_downstream, test_aggbymean_upstream_downstream]

    for i, dt in enumerate(data_list):

        ty = pd.concat([pd.Series(list(kawakami_predictions.keys())), pd.DataFrame(dt)], axis=1)

        column_names = ['id', 'class']
        column_names.extend([f'f_{i}' for i in range(1, ty.shape[1] - 1)])

        ty = ty.set_axis(column_names, axis=1, copy=False)

        ty.to_csv(path_or_buf=f'{save_dir}/{each_id}_{data_names[i]}_{TF}.csv.gz', index=False, compression='gzip')
    print(f'[INFO] Finished saving data for {each_id}')

    return(0)

In [10]:
collected = collect_modeling_data_for_kawakami(each_id=each_id, log_data=log_data, predictions_path=enformer_predictions_path, TF=TF, data_names=data_names, base_path=base_path, save_dir=save_dir)

#print(collected)

print(f'[INFO] Status: {collected} for {log_data.shape[0]} predictions for {each_id}.')

[INFO] Finished collecting 0 predictions for kawakami


ValueError: need at least one array to concatenate